## Develop this to have entrylist with highest entry lowest and average

## maybe entry from the average trade script

In [1]:
import numpy as np 
import pandas as pd 
from datetime import date
#Data Source (Pandas Data Reader)
from pandas_datareader import data
import requests 
import json
from dateutil.relativedelta import relativedelta


#Data Source(Pandas Data Reader)


In [2]:
#entrylist =[['ANKR/USDT',0.07684],['AVAX/USDT',57],['WAXP/USDT',0.3818],['REEF/USDT',0.01346],['LINK/USDT',14.15],['AR/USDT',30],['ATOM/USDT',23],['MIR/USDT',2.343],['ALGO/USDT',0.909],['ZRX/USDT',0.90],['XTZ/USDT',2.856],['KSM/USDT',163],]
#entrylist =[['ANKRUSDT',0.07684],['AVAXUSDT',57],['WAXPUSDT',0.3818],['REEFUSDT',0.01131],['LINKUSDT',14.15],['ARUSDT',30],['ATOMUSDT',23],['MIRUSDT',2.343],['ALGOUSDT',0.909],['ZRXUSDT',0.60],['XTZUSDT',2.856],['KSMUSDT',163],['NEARUSDT',14],['WINUSDT',0.0003138],['CELRUSDT',0.05063],['DENTUSDT',0.002459],['LRCUSDT',0.7047],['CRVUSDT',3.529],['IMXUSDT',2.709],['SCRTUSDT',5],['RENUSDT',0.2547],['SUSHIUSDT',4.522],['CAKEUSDT',11.24],['XLMUSDT',0.2109],['HNTUSDT',30],['OMGUSDT',4.5],['LINAUSDT',0.024],['MIRUSDT',2.343],['FLUXUSDT',1.523]]
entrylist =[['ANKRUSDT',0.05415],['AVAXUSDT',57],['WAXPUSDT',0.3174],['REEFUSDT',0.00930],['LINKUSDT',14.15],['ARUSDT',30],['ATOMUSDT',23],['MIRUSDT',2.343],['ALGOUSDT',0.909],['ZRXUSDT',0.50],['XTZUSDT',2.856],['KSMUSDT',150],['NEARUSDT',10.320],['WINUSDT',0.0003138],['DENTUSDT',0.002459],['LRCUSDT',0.677],['CRVUSDT',2.12],['IMXUSDT',1.30],['SCRTUSDT',5],['RENUSDT',0.3047],['SUSHIUSDT',3.5],['CAKEUSDT',11.24],['XLMUSDT',0.2109],['HNTUSDT',30],['OMGUSDT',4.5],['LINAUSDT',0.01759],['FLUXUSDT',1.523],['SRMUSDT',1.8],['VTHOUSDT',0.0031],['ANTUSDT',5.616],['RAYUSDT',2.5],['ROSEUSDT',0.24055],['MOVRUSDT',51.6],['TFUELUSDT',0.22],['GLMRUSDT',4.5],['COTIUSDT',0.2277],['ONEUSDT',0.13171],['VETUSDT',0.05138],['RUNEUSDT',3.709],['CELRUSDT',0.034063],['ONTUSDT',0.4652],['GALAUSDT',0.24794],['GRTUSDT',0.3164],['DOTUSDT',16.31],['ZILUSDT',0.04410],['FILUSDT',20],['SCUSDT',0.015],['MATICUSDT',1.4],['FTMUSDT',1.0],['BANDUSDT',3.6],['1INCHUSDT',1.6],['AUDIOUSDT',1.0]]
exposurelist =[['ANKRUSDT',311],['AVAXUSDT',2850],['WAXPUSDT',527],['REEFUSDT',603],['LINKUSDT',6000],['ARUSDT',2450],['ATOMUSDT',1863],['MIRUSDT',85],['ALGOUSDT',2735],['ZRXUSDT',245],['XTZUSDT',1200],['KSMUSDT',7800],['NEARUSDT',1360],['WINUSDT',285],['DENTUSDT',385],['LRCUSDT',660],['CRVUSDT',323],['IMXUSDT',269],['SCRTUSDT',601],['RENUSDT',684],['SUSHIUSDT',2730],['CAKEUSDT',4000],['XLMUSDT',368],['HNTUSDT',999],['OMGUSDT',765],['LINAUSDT',205],['FLUXUSDT',224],['SRMUSDT',587],['VTHOUSDT',413],['ANTUSDT',200],['RAYUSDT',100],['ROSEUSDT',634],['MOVRUSDT',500],['TFUELUSDT',244],['GLMRUSDT',400],['COTIUSDT',500],['ONEUSDT',1000],['VETUSDT',500],['RUNEUSDT',750],['CELRUSDT',500],['ONTUSDT',400],['GALAUSDT',650],['GRTUSDT',1700],['DOTUSDT',10000],['ZILUSDT',1000],['FILUSDT',2000],['SCUSDT',200],['MATICUSDT',8000],['FTMUSDT',2000],['BANDUSDT',750],['1INCHUSDT',1000],['AUDIOUSDT',700]]
startdate = '2021-01-01'
enddate = '2022-02-22'
enddate = pd.to_datetime('today')

interval = '1h'
#easyband = pd.read_csv('easyband.csv')
easyband = pd.read_csv('allbands.csv')

#Columns=['Indicator','ALLTIME','CLOSERATIO','SMA100','SMA55']
#easyband = pd.DataFrame(easyband,columns=['Indicator','ATH','CLOSERATIO','RED','ORANGE','GREEN','Current'])
# Indicator,RED,ORANGE,GREEN,Current
easyband = pd.DataFrame(easyband,columns=['Indicator','RED','ORANGE','GREEN','Current'])

#easyband = easyband.loc[:,['Indicator','GREEN','GREENEASY']]

#EASY=float(easyband.loc[(easyband.Indicator=='BANDUSDT')].SMA100.values)
#print(easyband)
#EASY[0]
#easyband

In [3]:
entries = pd.read_csv('entries.csv')

entries = pd.DataFrame(entries, columns =['Indicator', 'lowest_entry']) 

#entries = pd.DataFrame(entrylist, columns =['Indicator', 'lowest_entry'],dtype = float) 
entries
entries.lowest_entry.astype(float)
indicator_list = entries['Indicator']
#indicator_list
entries
entries.to_csv("entries.csv")

exposurelist = pd.read_csv('exposure.csv')
exposure = pd.DataFrame(exposurelist, columns =['Indicator', 'exposure']) 
exposure
exposure.exposure.astype(float)
#indicator_list
#entries

#entries = entries.merge(exposure ,how="left",on="Indicator")
#entries

0      311.0
1     3500.0
2      527.0
3      700.0
4     6000.0
       ...  
59     500.0
60    1000.0
61     600.0
62    1000.0
63     400.0
Name: exposure, Length: 64, dtype: float64

In [4]:
#whitelist=whitelist.lstrip()
#whitelist = indicator_list
#array = whitelist.split(",")
array = indicator_list
wl = []
bl = []
blacklist = ""
whitelist = ""
symbols =[]
for i in range(0 ,len(array)):
    if (array[i].find('DOWN') != -1):
        bl.append(str(array[i]).lstrip())
        #print ("Contains given substring ")
        blacklist += str(array[i])
    elif (array[i].find('UP') != -1):
        bl.append(str(array[i]).lstrip())
        #print ("Contains given substring ")
        blacklist += str(array[i])

    else:
        wl.append(str(array[i]).lstrip().replace("/",""))
        whitelist +=str(array[i]).lstrip()
        #print ("Doesn't contains given substring")
    

    
#symbols to be used to get the prices from binance url
symbols = np.array(wl)


## get the data 
import os

live = pd.to_datetime('today')
start = pd.to_datetime(startdate)
end = pd.to_datetime(enddate)

root_url = 'https://api.binance.com/api/v1/klines'
symbol = 'STEEMETH'
interval = '1d'
url = root_url + '?symbol=' + symbol + '&interval=' + interval
print(url)


directory = 'BN-main'
table = []
   
        
        

for symbol in symbols:
        url = root_url + '?symbol=' + symbol + '&interval=' + interval
        data = json.loads(requests.get(url).text)
        dflive = pd.DataFrame(data ,columns =['open_time',
              'o', 'h', 'l', 'c', 'v',
              'close_time', 'qav', 'num_trades',
              'taker_base_vol', 'taker_quote_vol', 'ignore'])
        dflive.open_time = pd.to_datetime(dflive['open_time'],unit='ms')
        dflive=dflive.loc[(dflive.open_time>start)]
        lastrow = dflive.iloc[-1]
        now= float(lastrow.c)
        alth = dflive.h.max()
        altl = dflive.l.min()
        
    
        ## lowest entiry
        lowest = float(entries.loc[(entries.Indicator==symbol)].lowest_entry.values)
        #print(lowest)
        easyseries = easyband.loc[(easyband.Indicator==symbol)].GREEN.values
        #print(easyseries)
        if easyseries.size>0:
                green = easyseries[0]
        else:
                green = 0
        oeasyseries = easyband.loc[(easyband.Indicator==symbol)].ORANGE.values
        #print(easyseries)
        if oeasyseries.size>0:
                orange = oeasyseries[0]
        else:
                orange = 0
 
        tuple =(symbol,alth,now,lowest,green,orange)
        table.append(tuple)
        #print("name is: ",symbol ,alth,altl)
        

table[0]
AllTime = pd.DataFrame(table,columns=['Indicator','ATH','Current','LOWEST','GREEN','ORANGE'])
AllTime.ATH=AllTime.ATH.astype(float)
#AllTime.ATL=AllTime.ATL.astype(float)
AllTime.Current=AllTime.Current.astype(float)
AllTime.LOWEST=AllTime.LOWEST.astype(float)
AllTime.GREEN=AllTime.GREEN.astype(float)
AllTime.ORANGE=AllTime.ORANGE.astype(float)




AllTime.dtypes

#AllTime['ALLTIME']=(AllTime['Current']/AllTime['ATH'])*100
#AllTime['MONTH']=(AllTime['Current']/AllTime['OMH'])*100
#AllTime['2MONTH']=(AllTime['Current']/AllTime['TMH'])*100
#AllTime['3MONTH']=(AllTime['Current']/AllTime['THMH'])*100
AllTime['DROP']=((AllTime['Current']-AllTime['LOWEST'])/AllTime['LOWEST'])*100
AllTime['GDROP']=((AllTime['GREEN']-AllTime['Current'])/AllTime['GREEN'])*100
AllTime['TDROP']=((AllTime['DROP']*AllTime['GDROP']))

#AllTime.sort_values(by=['ALLTIME'],ascending=True,inplace = True)
#AllTime.sort_values(by='ATH',ascending=False)
#AllTime.to_csv("alltimeHL.csv", mode="w")
#data.sort_values("Name", axis = 0, ascending = True,inplace = True, na_position ='last')

## Display the Data 
AllTime.shape



https://api.binance.com/api/v1/klines?symbol=STEEMETH&interval=1d


(64, 9)

In [5]:
## Display the Data coins run last  Month

#drop = AllTime.loc[(AllTime['MONTH'] < 35)]
#drop = AllTime.loc[(AllTime['MONTH'] < 70)]
#drop = AllTime.loc[(AllTime['MONTH'] < 80)&(AllTime['2MONTH'] < 70)&(AllTime['ALLTIME'] < 40)]
drop = AllTime.loc[(AllTime['Current'] <= AllTime['LOWEST'])]

#sort = drop.sort_values(by=['EASY'],ascending=False,inplace = True)
sort = drop.sort_values(by=['GDROP','DROP'],ascending=[False,True],inplace = True)
sort = drop.sort_values(by=['TDROP'],ascending=True,inplace = True)

#drop = drop.loc[(drop['ALLTIME'] < drop['3MONTH'])]
display = drop[['Indicator','TDROP','GREEN','DROP','GDROP','LOWEST','Current']]

#print(display.iloc[0:60,0:14])
#print(display.iloc[60:110,])
#print(display.iloc[110:160,])
print("\b Fix Lowest entry ")

display = display.merge(exposure ,how="left",on="Indicator")
#display1 = display.sort_values(by=['exposure'],ascending=True,inplace = True)


display.to_csv("fixentries.csv")

display

 Fix Lowest entry 


/var/folders/t3/7kfp7_l943j4z41f1p54xwdm0000gn/T/ipykernel_81717/1650543036.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sort = drop.sort_values(by=['GDROP','DROP'],ascending=[False,True],inplace = True)
/var/folders/t3/7kfp7_l943j4z41f1p54xwdm0000gn/T/ipykernel_81717/1650543036.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sort = drop.sort_values(by=['TDROP'],ascending=True,inplace = True)


,Indicator,TDROP,GREEN,DROP,GDROP,LOWEST,Current,exposure
0,LUNAUSDT,-7575.819062,31.257965,-81.918462,92.479997,13.0000,2.35060,4000.0
1,MIRUSDT,-6224.101188,0.536914,-92.503201,67.285252,2.3430,0.17565,85.0
2,PERPUSDT,-4929.935271,1.811667,-73.288288,67.267709,2.2200,0.59300,300.0
3,RAYUSDT,-2976.098201,1.160881,-72.600000,40.993088,2.5000,0.68500,100.0
4,GRTUSDT,-2472.711423,0.156824,-68.362832,36.170407,0.3164,0.10010,1700.0
...,...,...,...,...,...,...,...,...
58,GALAUSDT,inf,0.000000,-66.388235,-inf,0.1700,0.05714,650.0
59,MOVRUSDT,inf,0.000000,-59.643917,-inf,33.7000,13.60000,500.0
60,SCRTUSDT,inf,0.000000,-73.378378,-inf,3.7000,0.98500,601.0
61,FLUXUSDT,inf,0.000000,-70.584373,-inf,1.5230,0.44800,224.0


## Max entry Drop


In [6]:
## Max entry Drop
display = display.loc[:]
#display = display.loc[display.DROP <= -30]
display = display.loc[display.GDROP >= 0]

display.sort_values(by=['DROP'],ascending=True,inplace = True)

#display = display.loc[display.Current < display.GREEN*.9]

display

,Indicator,TDROP,GREEN,DROP,GDROP,LOWEST,Current,exposure
1,MIRUSDT,-6224.101188,0.536914,-92.503201,67.285252,2.343000,0.175650,85.0
0,LUNAUSDT,-7575.819062,31.257965,-81.918462,92.479997,13.000000,2.350600,4000.0
24,CAKEUSDT,-822.761105,3.320405,-73.754448,11.155410,11.240000,2.950000,4000.0
2,PERPUSDT,-4929.935271,1.811667,-73.288288,67.267709,2.220000,0.593000,300.0
5,FILUSDT,-2366.265132,8.024762,-72.900000,32.459055,20.000000,5.420000,2000.0
3,RAYUSDT,-2976.098201,1.160881,-72.600000,40.993088,2.500000,0.685000,100.0
11,SUSHIUSDT,-2126.598989,1.417286,-71.542857,29.724826,3.500000,0.996000,2730.0
6,MINAUSDT,-2321.070420,0.936810,-68.418367,33.924668,1.960000,0.619000,300.0
4,GRTUSDT,-2472.711423,0.156824,-68.362832,36.170407,0.316400,0.100100,1700.0
10,RENUSDT,-2181.685302,0.142121,-68.263866,31.959592,0.304700,0.096700,684.0


In [7]:
## GT drop
#print("\b Enries on easy band")

display1 = display.loc[:]
display1['GT']=display1['TDROP']*display1['exposure']
display1 = display1.loc[display1.GDROP >= 0]

display1.sort_values(by=['GT'],ascending=True,inplace = True)
display1

,Indicator,TDROP,GREEN,DROP,GDROP,LOWEST,Current,exposure,GT
0,LUNAUSDT,-7575.819062,31.257965,-81.918462,92.479997,13.000000,2.350600,4000.0,-3.030328e+07
18,MATICUSDT,-1401.222416,0.602405,-40.895522,34.263468,0.670000,0.396000,8000.0,-1.120978e+07
21,KSMUSDT,-1183.413501,64.004762,-56.086957,21.099621,115.000000,50.500000,7800.0,-9.230625e+06
8,AVAXUSDT,-2286.422246,30.504762,-49.909091,45.811739,33.000000,16.530000,3500.0,-8.002478e+06
11,SUSHIUSDT,-2126.598989,1.417286,-71.542857,29.724826,3.500000,0.996000,2730.0,-5.805615e+06
5,FILUSDT,-2366.265132,8.024762,-72.900000,32.459055,20.000000,5.420000,2000.0,-4.732530e+06
4,GRTUSDT,-2472.711423,0.156824,-68.362832,36.170407,0.316400,0.100100,1700.0,-4.203609e+06
38,DOTUSDT,-332.233379,7.896429,-50.800000,6.540027,15.000000,7.380000,10000.0,-3.322334e+06
39,DOTUSDT,-332.233379,7.896429,-50.800000,6.540027,15.000000,7.380000,10000.0,-3.322334e+06
24,CAKEUSDT,-822.761105,3.320405,-73.754448,11.155410,11.240000,2.950000,4000.0,-3.291044e+06


In [8]:
#display1['Value']=(display['DROP']/100)*display['exposure']
#display1

## Green Drop


In [9]:
## Green Drop
display = display.loc[:]
display = display.loc[display.GDROP >= 0]
display.sort_values(by=['GDROP'],ascending=False,inplace = True)

#display = display.loc[display.Current < display.GREEN*.9]

display

,Indicator,TDROP,GREEN,DROP,GDROP,LOWEST,Current,exposure
0,LUNAUSDT,-7575.819062,31.257965,-81.918462,92.479997,13.000000,2.350600,4000.0
1,MIRUSDT,-6224.101188,0.536914,-92.503201,67.285252,2.343000,0.175650,85.0
2,PERPUSDT,-4929.935271,1.811667,-73.288288,67.267709,2.220000,0.593000,300.0
16,FTMUSDT,-1493.918390,0.550936,-25.677419,58.180239,0.310000,0.230400,2000.0
13,ONEUSDT,-1789.797389,0.055581,-33.538462,53.365518,0.039000,0.025920,1000.0
20,ROSEUSDT,-1213.763064,0.097786,-24.219357,50.115413,0.064370,0.048780,634.0
8,AVAXUSDT,-2286.422246,30.504762,-49.909091,45.811739,33.000000,16.530000,3500.0
3,RAYUSDT,-2976.098201,1.160881,-72.600000,40.993088,2.500000,0.685000,100.0
19,ATOMUSDT,-1288.814200,10.560714,-33.435897,38.545823,9.750000,6.490000,1863.0
4,GRTUSDT,-2472.711423,0.156824,-68.362832,36.170407,0.316400,0.100100,1700.0


display = display1.loc[:]
#display['RED']=display['ORANGE']*2
display['Value']=(display['DROP']/100)*display['exposure']
sort = display.sort_values(by=['Value'],ascending=True,inplace = True)

#display['Ratio']=display['ATH']/display['Current']
#display['ORatio']=display['ORANGE']/display['Current']
#display = display.loc[display['Current'] <= display['GREEN']]
display


In [10]:
#del display ,sort, AllTime

In [11]:
#AllTime